In [92]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from glob import glob
import numpy as np

In [122]:
batch_size = 16

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
data_generator = train_datagen.flow_from_directory(
        './train/',  # this is the target directory
        target_size=(64, 64),  # all images will be resized to 150x150
        color_mode = 'grayscale', # rgb
        batch_size=batch_size,
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels


Found 25000 images belonging to 2 classes.


In [123]:
X = list()
y = list()
batch_index = 0

while batch_index <= data_generator.batch_index:
    data = data_generator.next()
    for batch_data_X,batch_data_y in zip(data[0],data[1]):
        X.append(batch_data_X)
        y.append(batch_data_y)
    batch_index = batch_index + 1
    if batch_index>1000:
        break
del data_generator
X = np.asarray(X)
y = to_categorical(y, num_classes=2)

In [124]:
X.shape

(16016, 64, 64, 1)

In [125]:
n_imgs = X.shape[0]
n_cols = X.shape[1]*X.shape[2]

In [126]:
#X = X.reshape(n_imgs, n_cols)

In [132]:
model = Sequential()
model.add(Conv2D(32,kernel_size=(3, 3), activation= 'relu',  input_shape=X.shape[1:]))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Flatten())

model.add(Dense(3,activation='relu'))

model.add(Dropout(0.2))
model.add(Dense(2,activation='softmax',input_shape= (n_cols,) ))

In [133]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [134]:
my_callback = EarlyStopping(patience=3)

In [135]:
model.fit(X, y,
        epochs=50,
        validation_split=0.2,
        callbacks =  [my_callback])
#model.save_weights('first_try.h5')  # always save your weights after training or during training

Train on 12812 samples, validate on 3204 samples
Epoch 1/50
 8032/12812 [=================>............] - ETA: 12s - loss: 0.6940 - acc: 0.4951

KeyboardInterrupt: 

In [131]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_19 (Conv2D)           (None, 62, 62, 32)        320       
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 20, 20, 32)        0         
_________________________________________________________________
flatten_9 (Flatten)          (None, 12800)             0         
_________________________________________________________________
dense_41 (Dense)             (None, 50)                640050    
_________________________________________________________________
dropout_7 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_42 (Dense)             (None, 2)                 102       
Total params: 640,472
Trainable params: 640,472
Non-trainable params: 0
_________________________________________________________________
